## Discrete Time Quantum Walks

### MIT iQuHACK 2020 | Team "Quantum Waddle"

This work explores the implementation of discrete-time quantum walks on a 1D and 2D chain as described in [1] using IBM's quantum computing platform. A useful tutorial referenced here can be found at [2].

### Contributors
Biswaroop Mukherjee, Carsten Robens, Maya Reese, Lamia Ateshian, Yiqi Ni, Enrique Mendez

### Introduction
Classical random walks are used widely in graph searching and optimization, with many important applications in route planning, game playing, physics, and finance. This motivates researchers to look at the quantum analogue of a random walk, a quantum walk. 

A simple random walk example consists of a 1D chain whose nodes are enumerated by integer values and connected to its two adjacent neighbors. A random walker in position $i$ steps to either $i+1$ or $i-1$ depending on the outcome of a coin flip. 

In the quantum mechanical analogue, we consider evolution of position on a lattice conditioned on the spin state of a spin-1/2 system.

<img src="figures/random_walk.png" width=700px>

### Motivation
One primary motivation is to determine whether quantum computers are faster at solving problems than classical computers.

Prior to 2003, exponential speedup was only demonstrated on algorithms that depended on the quantum Fourier transform. It's an interesting question to determine whether there is algorithmic variety in exponential speed up. It turns out that quantum walks can exhibit an exponential speed up[3].

In the paper[3] by Childs, *et al.*, they define the graph traversal problem. The idea is that if they implement graphs encoded via *so called* black box functions, they demonstrate that for certain complicated graphs, quantum walks can traverse the graph exponentially faster than any classical algorithm.
Our desire was to implement a quantum walk in IBM's quantum computer, starting in particularly simple cases and then work our way up to the full fledged case. As determining actual circuit configurations in a real quantum computer is a non-trivial problem we followed the work given in [1].

### Quantum Walk on a Lattice

The state of the system evolves with the application of two unitary operators:

1. A coin flip operator
2. A conditional shift operator

The state of our system $\vert \Psi \rangle$ is the product state of the coin and the position on the lattice
$\vert \Psi \rangle = \vert s \rangle \otimes \vert \psi \rangle$.

Coin space: $\vert s \rangle \in \mathcal{H}_C = \{ \vert \uparrow \rangle, \vert \downarrow \rangle \}$

Position space: $ \vert \psi \rangle = \mathcal{H}_P = \{ \sum_{i \in Z} \alpha_i \vert i \rangle \} $

#### Hadamard coin

For a walk on a chain, the coin flip operator is a $2 \times 2$ unitary matrix. A typical choice would be the Hadamard coin,

$H = \frac{1}{\sqrt 2} \begin{bmatrix} 1 & 1\\ 1 & -1 \end{bmatrix}$

#### Shift operator

The shift operator S is a position translating operator conditioned on the coin as the control qubit. It increments the position by one if the particle is in the spin up state, and decrements by one if the particle is in the spin down state:

$ S = \sum_{i \in Z} \mid i + 1 \rangle \langle i \mid \otimes \vert \uparrow \rangle \langle \uparrow \vert + \vert i - 1 \rangle \langle i \vert \otimes \vert \downarrow \rangle \langle \downarrow \vert $

#### Single time step
If the system is initialized in the state $\vert \Psi \rangle = \vert \uparrow \rangle \otimes \vert 0 \rangle$,

a single time step involves the application of a Hadamard coin flip

$\vert \uparrow \rangle \otimes \vert 0 \rangle \xrightarrow{H} \frac{1}{\sqrt 2} \big( \vert \uparrow \rangle + \vert \downarrow \rangle \big) \otimes \vert 0 \rangle $

followed by a conditional shift

$ \xrightarrow{S} \frac{1}{\sqrt 2} \big( \vert \uparrow \rangle \otimes \vert 1 \rangle  + \vert \downarrow \rangle \otimes \vert -1 \rangle \big) $

### Quantum Cycle Graph

A 1D cycle chain of 8 nodes is represented by the following figure:
<img src="figures/cycle_graph.png" width=400px>

A quantum walk on the cycle can be efficiently and straightforwardly implemented with a set of quantum gates consisting of Hadamard gates followed by conditional increment and decrement gates, described below.

### 1D Quantum Walk Circuit
This shows the quantum circuit for a 1D walk along a circular lattice [1]. A $2^n$-node circuit requires n qubits to encode the position and one coin.
<img src="figures/1d_circuit.png" width="600px">

### 2D Quantum Walk Circuit
The equivalent 2D circuit has n qubits each to encode the x and y positions, and two coins. At any given time step, the position can shift up, down, left, or right. [1]
<img src="figures/2d_circuit.png" width="600px">

### Increment and Decrement Gates

In [1]:
import numpy as np
import time
import networkx as nx
import matplotlib.pyplot as plt
import random
from qiskit import (QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer)
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import tkinter as tk
%matplotlib inline

In [2]:
n=3
def counts_to_prob_1d(counts):
    # Convert histogram counts to probability vector of size 1 x 2^n
    states = list(counts.keys())
    state_counts = list(counts.values())
    nshots = sum(state_counts)
    # Convert binary to integer index, reversing order as consistent w qiskit convention
    states_x = [int(s[::-1],2) for s in states]
    # Create array of probability values
    probabilities = np.zeros(2**n)
    probabilities[states_x] = state_counts
    probabilities /= nshots
    return probabilities

In [3]:
def increment_gate(circuit, qpos):
    n = len(qpos)
    for i in range(n):
        circuit.mct(qpos[i+1:], qpos[i], None, mode='noancilla')
def decrement_gate(circuit, qpos):
    n = len(qpos)
    for i in range(n):
        if i+1 < n: circuit.x(qpos[i+1:])
        circuit.mct(qpos[i+1:], qpos[i], None, mode='noancilla')
        if i+1 < n: circuit.x(qpos[i+1:])

In [4]:
seed = 12
random.seed(seed)
np.random.seed(seed)
def plotCircleGraph(propabilities):
    G = nx.Graph()
    colorarray = []
    # generate array of colors
    numProp = len(propabilities)
    for idx in range(numProp):
        colorarray.append([1-propabilities[idx],1-propabilities[idx],1])
    # generate graph
    for idx in range(numProp-1):
        G.add_edge(idx, idx+1)
    # add last edge
    G.add_edge(0,numProp-1)
    nx.draw(G,pos=nx.circular_layout(G),node_color = colorarray,cmap=plt.cm.Blues)

In [5]:
def f(x):
    simulator = Aer.get_backend('qasm_simulator')
    qpos = QuantumRegister(n,'qc')
    cpos = ClassicalRegister(n,'cr')
    circuit = QuantumCircuit(qpos, cpos)
    if x<0:
        for i in range(-x):
            decrement_gate(circuit, qpos)
    for i in range(x):
        increment_gate(circuit, qpos)
    # # Map the quantum measurement to the classical bits
    circuit.measure(qpos,cpos)
    # # Execute the circuit on the qasm simulator
    job = execute(circuit, simulator, shots=1000)
    # # Grab results from the job
    result = job.result()
    # # Returns counts
    counts = result.get_counts(circuit)
    prob = counts_to_prob_1d(counts)
    plotCircleGraph(prob)

### Circuit Representation

#### Increment Gate
<img src="figures/inc_only.png" width="auto">

#### Decrement Gate
<img src="figures/dec_only.png" width="auto">


### Visualization

In [6]:
interact(f, x=widgets.IntSlider(min=-10, max=10, step=1, value=0));

interactive(children=(IntSlider(value=0, description='x', max=10, min=-10), Output()), _dom_classes=('widget-i…

### One time step of a Hadamard Walk
A single time step of a Hadamard quantum walk combines the components into the circuit represented below:
<img src="figures/1step.png" width="auto">

## 1D and 2D Quantum Walk Simulation
The 1D and 2D quantum walks were implemented in the notebooks <a href="1D Walk on IBM.ipynb">1D Walk on IBM.ipynb</a> and <a href="2D Walk on IBM.ipynb">2D Walk on IBM.ipynb</a>. 

## Outlook

Having demonstrated the 1-D and 2-D case, we are ready to move on to more complicated quantum walks that actually exhibit a quantum speed up. Fortunately [3], has a circuit for implementing a transformed quantum walk problem, but the actual walking for the graph of interest needs to be implemented by a black box function. The paper doesn't illustrate how to implement the actual black box function, so we came up with an algorithmic way to implement any black box function.

Once that's done, it's straightforward to include the rest of the quantum gates necessary to implement exponentially sped up quantum walks on arbitrary quantum graphs. One more hack-a-thon should be enough to implement arbitrary quantum walks on a quantum computer!

## References
[1]: Douglas, B. L., & Wang, J. B. (2009). Efficient quantum circuit implementation of quantum walks. Physical Review A, 79(5), 052335.

[2]: Reitzner, D., Nagaj, D., & Buzek, V. (2012). Quantum walks. arXiv preprint arXiv:1207.7283.

[3]: Childs, A. M., Cleve, R., Deotto, E., Farhi, E., Gutmann, S., & Spielman, D. A. (2003, June). Exponential algorithmic speedup by a quantum walk. In Proceedings of the thirty-fifth annual ACM symposium on Theory of computing (pp. 59-68).